In [ ]:
# just execute this cell to have the proper functions imported
# the cell must run correctly to be able to run the assignement
#
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as si
from scipy import __version__ as scipy_version
answers = {}


### Risk-Neutral Black-Scholes SDE

In the risk-neutral measure, the Black-Scholes stochastic differential equation (SDE) for the stock price $ S_t $ is:

$$
dS_t = r S_t \, dt + \sigma S_t \, dW_t
$$

where:
- $ r $ is the risk-free interest rate,
- $ \sigma $ is the volatility of the stock,
- $ W_t $ is a standard Brownian motion.

---

### SDE for $ \ln(S_t) $

Using Itô's Lemma, we can derive the SDE for $ \ln(S_t) $. Let $ X_t = \ln(S_t) $. Then:

$$
dX_t = d(\ln(S_t)) = \left( r - \frac{1}{2} \sigma^2 \right) dt + \sigma \, dW_t
$$

This is a simpler SDE because it has no dependence on $ S_t $. The solution to this SDE is:

$$
X_t = X_0 + \left( r - \frac{1}{2} \sigma^2 \right) t + \sigma W_t
$$

where $ X_0 = \ln(S_0) $.

---

### Option Price $ C(T, K) $

The price of a European call option with maturity $ T $ and strike $ K $ is given by:

$$
C(T, K) = \mathbb{E} \left[ e^{-rT} \max(S_T - K, 0) \right]
$$

Under the risk-neutral measure, $ S_T $ is log-normally distributed:

$$
S_T = S_0 \exp \left( \left( r - \frac{1}{2} \sigma^2 \right) T + \sigma W_T \right)
$$

---

### Statistical Estimator $ \hat{C}(T, K) $

To estimate $ C(T, K) $ using Monte Carlo simulation, we can simulate $ N_{\text{sim}} $ paths of $ \ln(S_T(\omega_i)) $ and compute the average discounted payoff. Here's how:

1. **Simulate $ \ln(S_T(\omega_i)) $**:
   - Generate $ N_{\text{sim}} $ independent standard normal random deviates $ Z(\omega_i) \sim \mathcal{N}(0, 1) $.
   - For each $ Z(\omega_i) $, compute:
     $$
     \ln(S_T(\omega_i)) = \ln(S_0) + \left( r - \frac{1}{2} \sigma^2 \right) T + \sigma \sqrt{T} Z(\omega_i)
     $$

2. **Compute $ S_T(\omega_i) $**:
   - Exponentiate to get the stock price at time $ T $:
     $$
     S_T(\omega_i) = \exp \left( \ln(S_T(\omega_i)) \right)
     $$

3. **Compute the Payoff**:
   - For each $ S_T(\omega_i) $, compute the payoff:
     $$
     \text{Payoff}(\omega_i) = \max(S_T(\omega_i) - K, 0)
     $$

4. **Estimate $ C(T, K) $**:
   - Average the discounted payoffs:
     $$
     \hat{C}(T, K) = e^{-rT} \cdot \frac{1}{N_{\text{sim}}} \sum_{i=1}^{N_{\text{sim}}} \text{Payoff}(\omega_i)
     $$
     

In [ ]:
def black_scholes_mc_call_price(S0, K, T, r, sigma, param):
    np.random.seed(42)  # You can use any integer value as the seed
    # Step 1: Generate uniform random variables U
    U = np.random.uniform(0, 1, param["N_sim"])
    # Step 2: Transform U into standard normal random variables Z using the inverse CDF
    Z = si.norm.ppf(U)
    # Step 2: Simulate ln(S_T(\omega_i))
    ln_ST = np.log(S0) + (r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z
    # Step 3: Compute S_T(\omega_i)
    ST = np.exp(ln_ST)
    # Step 4: Compute payoffs
    payoffs = np.maximum(ST - K, 0)
    # Step 5: Estimate C(T, K)
    C_hat = np.exp(-r * T) * np.mean(payoffs)
    return C_hat

### Example usage
S0 = 100  # Initial stock price
K = 100   # Strike price
T = 1     # Time to maturity (1 year)
r = 0.05  # Risk-free rate
sigma = 0.2  # Volatility
N_sim = 100000  # Number of simulations
C_hat = black_scholes_mc_call_price(S0, K, T, r, sigma, {"N_sim":N_sim})
answers["call price estimator for 100000 mc steps is"] = C_hat
C_hat

### Black Scholes Closed-Form Formula 
$$
C(T, K) = S_0 \Phi(d_1) - K e^{-rT} \Phi(d_2)
$$

where:
- $ \Phi(\cdot) $ is the cumulative distribution function (CDF) of the standard normal distribution,
- $ d_1 $ and $ d_2 $ are given by:

$$
d_1 = \frac{\ln(S_0 / K) + (r + \sigma^2 / 2) T}{\sigma \sqrt{T}}
$$

$$
d_2 = d_1 - \sigma \sqrt{T}
$$

In [ ]:
def black_scholes_call_price(S0, K, T, r, sigma):
    np.random.seed(42)  # You can use any integer value as the seed
    # Calculate d1 and d2
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    # Calculate the call option price
    call_price = S0 * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)
    return call_price

C = black_scholes_call_price(S0, K, T, r, sigma)
answers["exact BSM price that we try to approximate is"] = C
C

### Convergence Study

The performance of the estimator $\hat{C}(T, K)$ can be compared to $C(T,K)$ using

$$
err = \frac{|\hat{C}(T, K)-C(T,K)|}{S_0}
$$

As $N_{sim}$ increases, we expect the estimator standard deviation to reduce with $1/\sqrt{N_{sim}}$

In [ ]:
nsims = (np.arange(5,300))**2
err = [(black_scholes_mc_call_price(S0, K, T, r, sigma, {"N_sim":nsim})-C)/S0 for nsim in nsims]
plt.plot(np.sqrt(nsims), np.log10(np.abs(err)))
plt.xlabel("Nsim^1/2")
plt.ylabel("log10 of abs estimation error")
plt.title("relative error")
plt.grid()
plt.show()
# relative error is np.log10(abs(Chat-C)/S0))
answers["we need what Nsim at the minimum to have a 1e-2 precision"] = 0 # approximate Nsim
answers["we need what Nsim at the minimum to have a 1e-3 precision"] = 0 # approximate Nsim
answers["numerical precision of 1e-5 seems hard to get with this method"] = "yes/no"

### Antithetic Sampling

**Antithetic sampling** is a variance reduction technique used in Monte Carlo simulations. It involves generating pairs of negatively correlated random samples to reduce the overall variance of the estimator. When sampling from a uniform distribution $ U \sim \text{Uniform}(0, 1) $, its antithetic counterpart is $ 1 - U $. These pairs are then transformed into standard normal variables $ Z $ and $ -Z $ using the inverse CDF (e.g., $ Z = \Phi^{-1}(U) $ and $ -Z = \Phi^{-1}(1 - U) $). By averaging the results from $ Z $ and $ -Z $, the estimator's variance is reduced because the errors in the two samples tend to cancel each other out. This method is particularly effective for symmetric distributions and improves computational efficiency without requiring additional random samples.

For a function $ f $ and uniform samples $ U_i $, the antithetic estimator is:

$$
\hat{C}_{\text{antithetic}} = \frac{1}{N} \sum_{i=1}^{N} \frac{f(Z_i) + f(-Z_i)}{2},
$$

where $ Z_i = \Phi^{-1}(U_i) $ and $ -Z_i = \Phi^{-1}(1 - U_i) $.

In [ ]:
def black_scholes_mc_call_price(S0, K, T, r, sigma, param):
    np.random.seed(42)  # You can use any integer value as the seed
    # Step 1: Generate uniform random variables U
    if param.get("antithetic")==True:
        U = np.random.uniform(0, 1, param["N_sim"]//2)
        U = np.hstack([U,1-U])
    else:
        U = np.random.uniform(0, 1, param["N_sim"])
    # Step 2: Transform U into standard normal random variables Z using the inverse CDF
    Z = si.norm.ppf(U)
    # Step 2: Simulate ln(S_T(\omega_i))
    ln_ST = np.log(S0) + (r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z
    # Step 3: Compute S_T(\omega_i)
    ST = np.exp(ln_ST)
    # Step 4: Compute payoffs
    payoffs = np.maximum(ST - K, 0)
    # Step 5: Estimate C(T, K)
    C_hat = np.exp(-r * T) * np.mean(payoffs)
    return C_hat

nsims = (np.arange(5,300))**2
err = [(black_scholes_mc_call_price(S0, K, T, r, sigma, {"N_sim":nsim})-C)/S0 for nsim in nsims]
erranti = [(black_scholes_mc_call_price(S0, K, T, r, sigma, {"N_sim":nsim,"antithetic":True})-C)/S0 for nsim in nsims]
plt.plot(np.sqrt(nsims), np.log10(np.abs(err)),label="standard")
plt.plot(np.sqrt(nsims), np.log10(np.abs(erranti)),label="antithetic")
plt.legend()
plt.xlabel("Nsim^1/2")
plt.ylabel("log10 of abs estimation error")
plt.title("error")
plt.show()
answers["is antithetic sampling better?"] = "yes/no" 
answers["how much of an order of magnitude change does antithetic sampling give?"] = 0 # this is 1 if error goes from 1e-3 to 1e-4
answers["numerical precision of 1e-5 seems hard to get with antithetic"] = "yes/no"

### Low Descrepancy Pseudo-Random Sequence

A **Sobol sequence** is a **low-discrepancy sequence** used in quasi-Monte Carlo methods for numerical integration and simulation. It generates points in a highly uniform manner across a multidimensional space, ensuring better coverage than pseudorandom numbers. Sobol sequences are deterministic and constructed using base-2 arithmetic, making them efficient and reproducible. They are particularly useful for reducing variance in Monte Carlo simulations, leading to faster convergence. Each point in the sequence depends on the previous ones, ensuring no clustering or gaps. Sobol sequences are widely used in finance, engineering, and computer graphics for their ability to provide accurate results with fewer samples.

In [ ]:
def black_scholes_mc_call_price(S0, K, T, r, sigma, param):
    np.random.seed(42)  # You can use any integer value as the seed
    # Step 1: Generate uniform random variables U
    nsim = param["N_sim"]//2 if param.get("antithetic")==True else param["N_sim"]
    if param.get("sobol")==True:
        U = si.qmc.Sobol(d=1).random(nsim)
    else:
        U = np.random.uniform(0, 1, nsim)
    if param.get("antithetic")==True:
        U = np.hstack([U,1-U])
    # Step 2: Transform U into standard normal random variables Z using the inverse CDF
    Z = si.norm.ppf(U)
    # Step 2: Simulate ln(S_T(\omega_i))
    ln_ST = np.log(S0) + (r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z
    # Step 3: Compute S_T(\omega_i)
    ST = np.exp(ln_ST)
    # Step 4: Compute payoffs
    payoffs = np.maximum(ST - K, 0)
    # Step 5: Estimate C(T, K)
    C_hat = np.exp(-r * T) * np.mean(payoffs)
    return C_hat

nsims = (np.arange(5,200))**2
err = [(black_scholes_mc_call_price(S0, K, T, r, sigma, {"N_sim":nsim,"sobol":True})-C)/S0 for nsim in nsims]
errantisobol = [(black_scholes_mc_call_price(S0, K, T, r, sigma, {"N_sim":nsim,"sobol":True,"antithetic":True})-C)/S0 for nsim in nsims]
erranti = [(black_scholes_mc_call_price(S0, K, T, r, sigma, {"N_sim":nsim,"sobol":False,"antithetic":True})-C)/S0 for nsim in nsims]
plt.plot(np.sqrt(nsims), np.log10(np.abs(err)),label="standard sobol")
plt.plot(np.sqrt(nsims), np.log10(np.abs(errantisobol)),label="antithetic sobol")
plt.plot(np.sqrt(nsims), np.log10(np.abs(erranti)),label="antithetic")
plt.legend()
plt.xlabel("Nsim^1/2")
plt.ylabel("log10 of abs estimation error")
plt.title("error for sobol sequence")
plt.show()
answers["is sobol better?"] = "yes/no"
answers["numerical precision of 1e-5 seems hard to get with sobol"] = "yes/no"
answers["does it look interesting to combine sobol and antithetic?"] = "yes/no"
answers["how much of an order of magnitude improvement in error do we get"] = 0 # this would be 2 for going from 1e-6 to 1e-8

### Checking the Numpy Gaussian Random Variable Generator
The numpy library can generate gaussian random variables directly. 

How good is convergence with these?


In [ ]:
def black_scholes_mc_call_price(S0, K, T, r, sigma, param):
    np.random.seed(42)  # You can use any integer value as the seed
    # Step 1: Generate uniform random variables U
    nsim = param["N_sim"]//2 if param.get("antithetic")==True else param["N_sim"]
    if param.get("numpy")==True:
        Z = np.random.normal(0, 1, nsim)
    else:
        if param.get("sobol")==True:
            U = si.qmc.Sobol(d=1).random(nsim)
        else:
            U = np.random.uniform(0, 1, nsim)
        # Step 2: Transform U into standard normal random variables Z using the inverse CDF
        Z = si.norm.ppf(U)
    if param.get("antithetic")==True:
        Z = np.hstack([Z,-Z])
    # Step 2: Simulate ln(S_T(\omega_i))
    ln_ST = np.log(S0) + (r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z
    # Step 3: Compute S_T(\omega_i)
    ST = np.exp(ln_ST)
    # Step 4: Compute payoffs
    payoffs = np.maximum(ST - K, 0)
    # Step 5: Estimate C(T, K)
    C_hat = np.exp(-r * T) * np.mean(payoffs)
    return C_hat

err = [(black_scholes_mc_call_price(S0, K, T, r, sigma, {"N_sim":nsim,"sobol":True})-C)/S0 for nsim in nsims]
errnumpy = [(black_scholes_mc_call_price(S0, K, T, r, sigma, {"N_sim":nsim,"numpy":True})-C)/S0 for nsim in nsims]
plt.plot(np.sqrt(nsims), np.log10(np.abs(err)),label="standard sobol")
plt.plot(np.sqrt(nsims), np.log10(np.abs(errnumpy)),label="numpy gauss")
plt.legend()
plt.xlabel("Nsim^1/2")
plt.ylabel("log10 of abs estimation error")
plt.title("error for sobol sequence")
plt.show()
answers["is sobol better than numpy guassian?"] = "yes/no"
answers["numerical precision of 1e-5 seems hard to get with numpy"] = "yes/no"


### Black-Scholes-Merton (BSM) SDE for $ n $ Stocks

For $ n $ stocks $ S^i_t $ ($ i = 1, \dots, n $), the **risk-neutral BSM SDE** is:

$$
dS^i_t = r S^i_t \, dt + \sigma_i S^i_t \, dW^i_t,
$$

where:
- $ r $ is the risk-free rate,
- $ \sigma_i $ is the volatility of stock $ i $,
- $ W^i_t $ is a standard Brownian motion for stock $ i $,
- The Brownian motions are correlated: $ dW^i_t \, dW^j_t = \rho_{ij} \, dt $, where $ \rho_{ij} $ is the correlation between $ W^i_t $ and $ W^j_t $.

---

### SDE for $ \ln(S^i_t) $

Using Itô's Lemma, the SDE for $ \ln(S^i_t) $ is:

$$
d\ln(S^i_t) = \left( r - \frac{1}{2} \sigma_i^2 \right) dt + \sigma_i \, dW^i_t.
$$

Integrating this, we get:

$$
\ln(S^i_T) = \ln(S^i_0) + \left( r - \frac{1}{2} \sigma_i^2 \right) T + \sigma_i W^i_T.
$$

---

### Covariance Matrix $ \Sigma $ of $ \ln(S^i_T) $

The covariance matrix $ \Sigma $ of $ \ln(S^i_T) $ is an $ n \times n $ matrix where:

$$
\Sigma_{ij} = \text{Cov}(\ln(S^i_T), \ln(S^j_T)) = \sigma_i \sigma_j \rho_{ij} T.
$$

The diagonal elements are:

$$
\Sigma_{ii} = \text{Var}(\ln(S^i_T)) = \sigma_i^2 T.
$$

---

### Monte Carlo Estimator $ \hat{P} $ for $ P = \mathbb{E}[\max(K - \min_i(S^i_T), 0)] $

To estimate the payoff $ P = \mathbb{E}[\max(K - \min_i(S^i_T), 0)] $, we can use Monte Carlo simulation as follows:

1. **Simulate Correlated Log-Normal Stock Prices**:
   - Generate correlated Brownian motions $ W^i_T $ using the covariance matrix $ \Sigma $.
   - Compute $ \ln(S^i_T) $ for each stock.
   - Exponentiate to get $ S^i_T $.

2. **Compute the Payoff**:
   - For each simulation $ \omega_j $, compute the minimum of the stock prices $ \min_i(S^i_T(\omega_j)) $.
   - Compute the payoff $ \max(K - \min_i(S^i_T(\omega_j)), 0) $.

3. **Average the Payoffs**:
   - The Monte Carlo estimator is the average of the payoffs across all simulations:

$$
\hat{P} = \frac{1}{N_{\text{sim}}} \sum_{j=1}^{N_{\text{sim}}} \max\left(K - \min_i(S^i_T(\omega_j)), 0\right).
$$

---

### Generating Correlated Normal Random Variables

To generate correlated normal random variables $ Z $ with covariance matrix $ \Sigma $, we use the **Cholesky decomposition** of $ \Sigma $:

1. **Cholesky Decomposition**:
   - Decompose $ \Sigma $ as $ \Sigma = A A^\top $, where $ A $ is a lower triangular matrix.

2. **Generate IID Normal Random Variables**:
   - Generate $ G \sim \mathcal{N}(0, I) $, where $ I $ is the identity matrix.

3. **Compute Correlated Normal Random Variables**:
   - Transform $ G $ using $ Z = A G $. The covariance of $ Z $ is $ \Sigma $.

---


### Explanation of the Code:
1. **Covariance Matrix**:
   - $ \Sigma $ is computed using the outer product of volatilities and the correlation matrix.

2. **Cholesky Decomposition**:
   - The Cholesky factor $ A $ is computed using `scipy.linalg.cholesky`.

3. **Random Variables**:
   - Uniform random variables $ U $ are transformed into standard normal variables $ G $ using the inverse CDF (`norm.ppf`).
   - Correlated normal variables $ Z $ are generated as $ Z = G A^\top $.

4. **Stock Prices**:
   - Log stock prices $ \ln(S_T) $ are simulated using the BSM formula.
   - Stock prices $ S_T $ are obtained by exponentiating.

5. **Payoff Calculation**:
   - The minimum of the stock prices $ \min_i(S^i_T) $ is computed for each simulation.
   - The payoff $ \max(K - \min_i(S^i_T), 0) $ is averaged to get the Monte Carlo estimator $ \hat{P} $.

---

### Formula for $ \hat{P} $:

The Monte Carlo estimator $ \hat{P} $ is given by:

$$
\hat{P} = \frac{1}{N_{sim}} \sum_{j=1}^{N_{sim}} \max\left(K - \min_i(S^i_T(\omega_j)), 0\right).
$$

This formula averages the payoffs across all simulations to estimate the expected value of the option payoff.

Let me know if you need further clarification!


In [ ]:
from scipy.linalg import cholesky

def is_positive_semi_definite(matrix):
    """
    Check if a matrix is positive semi-definite by verifying that all eigenvalues are non-negative.
    """
    eigenvalues = np.linalg.eigvals(matrix)
    return np.all(eigenvalues >= 0)

def montecarlo_worstput(S0, sigma, corr, K, r, T, nsim):
    n = len(S0)  # Number of stocks

    # Build the correlation matrix rho
    rho = np.eye(n)  # Identity matrix (1 on diagonal)
    rho[np.triu_indices(n, k=1)] = corr  # Set upper triangular off-diagonal elements
    rho[np.tril_indices(n, k=-1)] = corr  # Set lower triangular off-diagonal elements

    # Covariance matrix Σ
    Sigma = np.outer(sigma, sigma) * rho * T
    if not is_positive_semi_definite(Sigma):
        raise Exception(f"ERR: montecarlo_worstput corr={corr} leads to non sdp covariance")

    # Cholesky decomposition of Σ
    A = cholesky(Sigma, lower=True)

    # Generate uniform random variables U with shape (nsim,n)
    np.random.seed(42)  # You can use any integer value as the seed
    U = np.random.uniform(0, 1, (nsim,n))

    # Transform U to standard normal random variables G
    G = si.norm.ppf(U)

    # Generate correlated normal random variables Z
    Z = G @ A.T

    # Simulate log stock prices ln(S_T)
    ln_ST = np.log(S0) + (r - 0.5 * sigma**2) * T + Z

    # Compute stock prices S_T
    ST = np.exp(ln_ST)

    # Compute the minimum of the stock prices for each simulation
    min_ST = np.min(ST, axis=1)

    # Compute the payoff max(K - min_ST, 0)
    payoffs = np.maximum(K - min_ST, 0)

    # Monte Carlo estimator P_hat
    P_hat = np.exp(-r * T) * np.mean(payoffs)
    return P_hat

# Parameters
n = 3  # Number of stocks
T = 1  # Time to maturity
r = 0.05  # Risk-free rate
sigma = np.array([0.2, 0.3, 0.25])  # Volatilities
corr = 0.5  # Correlation coefficient (off-diagonal)
K = 100  # Strike price
S0 = np.array([100, 100, 100])  # Initial stock prices
nsim = 100000  # Number of simulations

# Run the Monte Carlo simulation
P_hat = montecarlo_worstput(S0,sigma,corr,K,r,T,nsim)
answers["worst off put price estimator for 100000 mc steps is"] = P_hat
P_hat

In [ ]:
P_hat = {}
for nsim in np.arange(10,200)**2:
    P_hat[nsim] = montecarlo_worstput(S0,sigma,corr,K,r,T,nsim)
plt.plot(np.sqrt(list(P_hat.keys())),P_hat.values())
plt.xlabel("Nsim^1/2")
plt.ylabel("estimator")
plt.title("price convergence for worst off put")
plt.show()
answers["worst off put price estimator variance seems to dampen for sqrt(Nsim) greater than"] = 0 # put recommended value

In [ ]:
P_hat = {}
for corr in np.linspace(-1,1,50):
    try:
        P_hat[corr] = montecarlo_worstput(S0,sigma,corr,K,r,T,nsim)
    except Exception as e:
        print(e)
plt.plot(P_hat.keys(),P_hat.values())
plt.xlabel("corr")
plt.ylabel("MC Price")
plt.title("worst-off Put price")
plt.show()
answers["how is worst-off put price impacted by correlation"] = "up/down" 

In [ ]:
# you need to edit this cell to edit MYEMAIL and MYKEY. running this cell will submit your answers
course = "mafs5330"
# import config
# config.createhash(answers,course)
# MYEMAIL = config.profemail[course]
# MYKEY = config.profapikey[course]
# edit line below when ready to post the answer, http status_code for succesful submission is 200
MYID  = 'rshao@connect.ust.hk' # <- EDIT email address that received api key from <mail.validation@mg.quantfinance.club>'
MYKEY = '46bb4348b44334e3277327c46999c668bc7db49a02fae092d32348136f5f3ad5'
import requests,json,time
def post_answer(question, answer):
    time.sleep(0.01)
    questionhash = {"call price estimator for 100000 mc steps is": "87d97f44f0a8764df28d4a7cd95dd7e9d4d698361cbe3ac0393749f556e03f7f", "exact BSM price that we try to approximate is": "461d0b79a9ca063afa397bc0a78aaefdf03bef0e9242a22d63ac027e34ab39dc", "we need what Nsim at the minimum to have a 1e-2 precision": "a6e4ac292681c5469c78e465eb783e4307a9d475ed5edd7b0cd3df0022ebeef4", "we need what Nsim at the minimum to have a 1e-3 precision": "641c511f77d70e9e68223becbbe3fe27c101549d97b664a35eac186c8ccf3208", "numerical precision of 1e-5 seems hard to get with this method": "e73d8180430cfe5a79e6ff2403ce0e2ce3f00c7c91d687bde923e469e0a2bd08", "is antithetic sampling better?": "fe64c006c4ab583d85c1a2b987c63ece9cc0625c9b7b0a087cf4e7ebb7b7d8ad", "how much of an order of magnitude change does antithetic sampling give?": "f47755597e3e3f33d0495833e481b6a09ab75e60170f3214026d26c3a52fab3f", "numerical precision of 1e-5 seems hard to get with antithetic": "656a69fc73d872ba43db7c5822a74e78e5932f9792e230f360f47a22ebf8f104", "is sobol better?": "fb2ad59e5625f15dfe779346226fddccf35f3c53177eef35a3697715a581d420", "numerical precision of 1e-5 seems hard to get with sobol": "5f33eaee60d0a8b6f507bfcf94517158ec427ac4bd167e8651932151d7541d86", "does it look interesting to combine sobol and antithetic?": "ae66c42fd17f34559445a1ca0edb3e9120f5d736b3c67a497588fb7180909c42", "how much of an order of magnitude improvement in error do we get": "2357eb8ef14072df0d86e994616ab3f3114a71d6d1067afa52d716dbb39450c1", "is sobol better than numpy guassian?": "9392ccc0eed161693a912f3528aeceeb1a17ac6f00cd5e5c5208e776e249bc21", "numerical precision of 1e-5 seems hard to get with numpy": "e9a9194564c5f3571eacd00deaa67c5206d470af163cb68eadc7f0a4e4536643", "worst off put price estimator for 100000 mc steps is": "c2b28106a1674f7e84692b77b8aaa4d30d8b66262d6abf627b6a840ba6f53d21", "worst off put price estimator variance seems to dampen for sqrt(Nsim) greater than": "3cbe451428197c2338ffed7f15de6dfc29ee8706ffbec9a78f1b5eb966ac9f37", "how is worst-off put price impacted by correlation": "4b01b7045a01754dc0b080899ed7cfb833d13aa647a18a562288ec28c6952c77"}
    return requests.post(f"https://www.quantfinance.club/{course}",data={'user':MYEMAIL,'mykey':MYKEY,'questionid':question,'answer':answer,'questionkey':questionhash[question]})
nbAnswers = 17
if len(answers)!=nbAnswers:
    raise Exception(f"There are {len(answers)} answers, should be {nbAnswers}. You can comment this check if you want to submit partial answers.")
for q,a in answers.items():
    x = post_answer(q, a)
    if x.status_code!=200:
        if x.status_code==401:
            raise Exception("API Key Authentication failed: please make sure the variables MYEMAIL and MYKEY are set")
        raise Exception("submission error with code:",x.status_code)
    else:
        print("submitted answer to %s=%s" % (q,str(a)))
print(f"check your latest submitted answers on:\nhttps://www.quantfinance.club/{course}_myanswers?email={MYEMAIL}&apikey={MYKEY}")
